#  Routing with IPv4
<i>Adapted for use with FABRIC from [IPv4 Routing Assignment](https://groups.geni.net/geni/wiki/GENIEducation/SampleAssignments/IPRouting/Procedure)</i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

This is the second step in this assignment. To go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb) 
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/IPV4%20routing/figures/IPV4Top.png?raw=true">
<br>The goal of this exercise is to setup the routing as indicated in the figure above; i.e. packets from A sent to IP address 192.168.2.12 on node C should be routed via node B. In order to create this routing behavior you will need to modify the routing tables in your nodes using the [linux route command](https://www.hscripts.com/tutorials/linux-commands/route.html)

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = 'IPv4_Routing'
try:
    slice = fablib.get_slice(slice_name)   
    slice.list_nodes()
except Exception as e:
    print(f"Get Slices Fail: {e}")

## 2. Guided Experiment
### 2.1 Routing Tables
The goal of this assignment is to understand what the effects of routing commands are and understand what the routing table represents. In this assignment you will set up the routing in the nodes as it was indicated in the figure below. Packets from A sent to IP address 192.168.2.12 on node C should be routed via node B. In order to create this routing behavior you will need to modify the routing tables in your nodes using the [linux route command](https://www.hscripts.com/tutorials/linux-commands/route.html)

#### The Route Command Introduction
The Linux route command follows the following general structure: `route {operation} {Destination} {Gateway} {InterfaceGateway}` knowing this, if you run `route` or `route -n` you will be able to display the current routing table. The routing table will display all of the available connections "routes'' that this host is connected to and their attributes. The following are the descriptions of what each of the columns represent:
    - **Destination** This is the IP address of the host's connection, you can use this IP to connect with the other host
    - **Gateway** This is the 'Hop' IP, if a host would like to communicate with another host, but does not have a direct connection, you can add the IP of a host who shares both of the nodes as a connection and route through this intermediary host to form a connection between yourself and the previously unreachable host
    - **Genmask** If you decide to add a network you will have to add a subnet mask that represents the available ips this connection can form, for example 11.0.0.0/16 would get attributed the net mask of 255.255.0.0 because you would be able to ping from 11.0.0.0 all the way to the ip address 11.0.255.255.
    - **Flags** There exist 3 flags when using the route command that represent the current status of the route: "U" means that the route is up and available, "H" means that the target is a host, "G" means that the target is a gateway.
    - **Iface** This column states what device the route is connected to.

The route command can take some arguments to add and delete routes; the following section will walk you through how to use each parameter.
1. Operation *Required*
    - **add** This flag asks the route command to *create* a new route. Keep in mind you can not modify an existing route, if you want to modify you will need to delete it first and then create a new one.
    - **del** This flag asks the route command to *delete* an existing route
2. Destination *Required*
    - **{IP}** If you only provide an IP the route command will treat the IP as if it contained the -host flag
    - **{Flag} {IP}** The destination section requires a flag and an IP to be completed. The IP in this section is the destination  IP of the host that you want to route to.
   	 - **-host {IP}** The host tag asks the route command to route to a single host.
   	 - **-net {IP} netmask {MASK}** The "net" tag asks the route command to create a connection to a range of possible connections. The MASK refers to a section of the ip that remains static when creating the range as described above.
3. Gateway *Optional*
    - **-gw {IP}**  If you want to route through another connection that you already have, this is where you would add the IP of the host you share in common.
4. InterfaceGateway *Required*
    - **dev {Device}** In this section you need to provide the name of the device this connection will use to route, the device must be connected to the destination host or to the node that is shared with the destination host


## 3. Assignment
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/IPV4%20routing/figures/ExerciseIPv4.png?raw=true"><br>

1. What happens when you traceroute from A to IP address 192.168.2.12 before you setup the static routes? Why?
	(Include the output of the traceroute in your writeup)
2. Setup the routing from A to 192.68.2.12 so that it goes through B. Was it enough to just modify the routing tables? What else did you need to change in order for the traffic to flow? Ensure that you have connectivity by running a ping from A to 192.168.2.12
	(In your writeup include all the commands you ran and a screenshot of the routing configuration)
3. What happens when you traceroute from A to IP address 192.168.2.12 after you setup the static routes?
	(In your writeup include a screenshot of the traceroute output.)

## Cleanup Resources

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")